In [ ]:
# Notebook 2: Supervised learning on popularity

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance

sns.set(style="whitegrid")

# Load cleaned data
df = pd.read_csv("data_cleaned.csv")

# Features and target
feature_cols = [
    "acousticness", "danceability", "duration_ms", "energy",
    "instrumentalness", "liveness", "loudness", "speechiness",
    "tempo", "valence", "key", "mode", "count",
    "duration_min", "is_instrumental", "is_acoustic",
    "main_genre_encoded"
]

feature_cols = [c for c in feature_cols if c in df.columns]
target_col = "popularity"

data = df.dropna(subset=[target_col] + feature_cols).copy()
df_sample = data.sample(20000, random_state=42)

X = df_sample[feature_cols]
y = df_sample[target_col]

print("Number of samples:", len(X))
print("Features used:", feature_cols)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

# Baseline: predict mean popularity
y_mean = np.full_like(y_test, y_train.mean(), dtype=float)
baseline_rmse = np.sqrt(mean_squared_error(y_test, y_mean))
baseline_mae = mean_absolute_error(y_test, y_mean)
print("\nBaseline model (mean popularity):")
print(f"RMSE: {baseline_rmse:.3f}")
print(f"MAE : {baseline_mae:.3f}")

# Compare models
models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "KNN_10": KNeighborsRegressor(n_neighbors=10)
}

rows = []

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    rows.append([name, rmse, mae, r2])

results_df = pd.DataFrame(rows, columns=["Model", "RMSE", "MAE", "R2"])
print("\nModel comparison:")
print(results_df.sort_values("RMSE"))

plt.figure(figsize=(8, 5))
sns.barplot(data=results_df, x="Model", y="RMSE")
plt.title("RMSE by model")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Cross validation on Random Forest
rf_base = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
cv_scores = cross_val_score(rf_base, X, y, cv=5, scoring="r2")
print("\nRandom Forest CV R2:", cv_scores)
print("Mean CV R2:", cv_scores.mean())

# Simple hyperparameter tuning
param_grid = {
    "n_estimators": [200],
    "max_depth": [None, 20],
}

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring="r2",
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

print("\nBest params:")
print(grid.best_params_)
print("Best CV R2:", grid.best_score_)

best_rf = grid.best_estimator_

# Final evaluation
y_pred = best_rf.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nTuned Random Forest on test:")
print(f"RMSE: {rmse:.3f}")
print(f"MAE : {mae:.3f}")
print(f"R2  : {r2:.3f}")
print(f"RMSE gain vs baseline: {baseline_rmse - rmse:.3f}")

# Predicted vs actual
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.3)
plt.plot([0, 100], [0, 100], "r-")
plt.xlabel("Actual popularity")
plt.ylabel("Predicted popularity")
plt.title("Predicted vs actual")
plt.tight_layout()
plt.show()

# Residuals
residuals = y_test - y_pred

plt.figure(figsize=(8, 4))
sns.histplot(residuals, bins=50, kde=True)
plt.title("Residuals")
plt.xlabel("Residual")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
plt.scatter(y_pred, residuals, alpha=0.3)
plt.axhline(0, color="r")
plt.xlabel("Predicted popularity")
plt.ylabel("Residual")
plt.title("Residuals vs predicted")
plt.tight_layout()
plt.show()

# Permutation importance
perm = permutation_importance(
    best_rf, X_test, y_test,
    n_repeats=10,
    random_state=42,
    n_jobs=-1
)

importances = pd.Series(perm.importances_mean, index=feature_cols).sort_values()

print("\nPermutation feature importance:")
print(importances.sort_values(ascending=False))

plt.figure(figsize=(8, 10))
importances.tail(15).plot(kind="barh")
plt.title("Feature importance (top 15)")
plt.xlabel("Mean decrease in R2")
plt.tight_layout()
plt.show()


Number of samples: 20000
Features used: ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'key', 'mode', 'count', 'duration_min', 'is_instrumental', 'is_acoustic', 'main_genre_encoded']
Train shape: (16000, 17) Test shape: (4000, 17)

Baseline model (mean popularity):
RMSE: 22.495
MAE : 19.123
